## Setup and Load Data

### 1. Setup Python Environment

This cell installs the required Python packages (`rpy2`) for running R code in this notebook.

In [ ]:
import sys
try:
    import rpy2
except ImportError:
    !{sys.executable} -m pip install rpy2

This cell loads the required Python packages (`rpy2`) for running R code in this notebook.

In [2]:
%load_ext rpy2.ipython

### 2. Load Data

In [5]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [ ]:
data = pd.read_excel("data/personal_finance_dataset.xlsx", sheet_name = "datathon_finance")
dictionary = pd.read_excel("data/personal_finance_dataset.xlsx", sheet_name = "dictionary")

## Cleaning the Dataset

### 1. Inspecting the Dataset
We inspect the dataset using shape, head, info, and describe(). All 19 columns and 16,241 rows have non-null values, indicating there are no missing entries.

In [ ]:
data.shape
data.head()
data.info()
data.describe()

data.isna().sum() # Check for missing values

### 2. Filter Age Group
Keep only respondents aged 18–54 (PAGEMIEG 1–4).

In [ ]:
# Remove the group ages not in 18-54
data = data[data['PAGEMIEG'].isin([1, 2, 3, 4])]

### 3. Classify Variables
We extract lists of binary, categorical, and continuous columns from the data dictionary.

In [ ]:
# Find binary columns
binary_cols = dictionary[dictionary['Type'] == 'Binary']['Variable Name'].tolist()
print(binary_cols)

# Find categorial columns
categorial_cols = dictionary[dictionary['Type'] == 'Categorical']['Variable Name'].tolist()
print(categorial_cols)

# Find continuous columns
continuous_cols = dictionary[dictionary['Type'] == 'Continuous']['Variable Name'].tolist()
print(continuous_cols)

### 4. Check for Unexpected Values
We check for outliers to decide what to do with those affected rows.

In [ ]:
# Binary columns
for col in binary_cols:
    print(col, sorted(data[col].unique()))

In [ ]:
# Categorical columns
for col in categorial_cols:
    print(col, sorted(data[col].unique()))

In [ ]:
# Continuous columns
for col in continuous_cols:
    print(col)
    print(data[col].describe())
    print(sorted(data[col].unique())[:10], '...', sorted(data[col].unique())[-10:])
    print()

After inspecting each column, we see there are no unexpected values, which are given from the dictionary.

### 5. Add necessary variables to the dataset
We add or transform variables needed for analysis, such as:
- Encoding categorical and binary variables into numeric form suitable for clustering.
- Scaling continuous financial variables to ensure all features are on a comparable scale.
- Creating any derived variables that summarize or combine information, if relevant (e.g., total debt, net assets).

## Identifying Key Predictors of Financial Stress and Stability

### Load Libraries in R

In [4]:
pip install "pandas[excel]" numpy matplotlib seaborn scikit-learn


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%%R
library(readxl)
library(tidyverse)

dataset <- read_excel("data/personal_finance_dataset.xlsx", sheet = "datathon_finance")
print(as.data.frame(head(dataset, 20), width = Inf))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.2
✔ purrr     1.2.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
   PAGEMIEG PATTCRU PATTSITC PATTSKP PEDUCMIE PEFATINC PFMTYPG PFTENUR PLFFPTME
1         6       4        3       2        4   115525       4       1        3
2         7       4        3       2        9   121300       2       1        3
3         5       4        3       2        4    19175       2       1        3
4         6       3        3       2        4   147425       2       2        2
5         4       4        3       2        3    56750       1       3      

Performing forward stepwise selection to determine the best model to predict the net worth of an individual.

## Building a segmentation model using clustering to find distinct financial personas
Doing segmentation using k-means clustering.